# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил: Дмитрий Монахов

### Общая информация

__Дата выдачи:__ 08.04.2020

__Дедлайн:__ 23:59 22.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\min(\text{points}, 21)  \times 10 / 21,
$$

где points — количество баллов за домашнее задание, которое вы набрали. Максимальное число баллов, которое можно получить за решение данного домашнего задания — 24, все баллы сверх 21 идут в бонус (таким образом, за данное домашнее задание можно получить 3 бонусных балла). Накопленные бонусные баллы можно будет потом распределять по другим домашним заданиям и проверочным (+1 бонусный балл = +1 к оценке за домашнее задание/проверочную).

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-4: zG1cIyT

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [76]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook

In [77]:
# чтобы видеть проход по итерациям, можно использовать библиотеку tqdm
# она работает примерно так:
from tqdm import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.83it/s]


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - `'title'`
 - `'description'`
 - `'Category_name'`
 - `'Category'`

Имеется информация об объектах 50 классов.
Задача: по новым объектам (`'title'`, `'description'`) предсказать `'Category'`.
(Очевидно, что параметр `'Category_name'` для предсказания классов использовать нельзя)

In [78]:
data = pd.read_csv("avito_data.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\r\n /\r\nАртикул: 1...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [79]:
data.shape

(30000, 4)

In [80]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [82]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\r\n/\r\n Данная цена только для подписчиков Instagram: iQmac/\r\n/\r\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\r\nСаженец/ корень 100р/\r\nРастут у нас более 70 лет/\r\nРозовые, бордовые и белые/\r\nНа фото цветы 2018г

In [83]:
y_train[:5]

array([ 27,  20,  84, 106,  27], dtype=int64)

### Токенизация (0.5 балла)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации. В данном задании мы будем использовать `WordPunctTokenizer`.

In [84]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже.

In [85]:
def preprocess(text: str, tokenizer) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    return ' '.join(tokenizer.tokenize(text.lower()))
    # your code here

In [86]:
assert preprocess(text, tokenizer) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте `'title'` и `'description'` в `train` и `test`.

In [87]:
X_test = np.array(list(map(lambda x: np.array([preprocess(x[0], tokenizer=tokenizer), 
                                        preprocess(x[1], tokenizer=tokenizer)]), list(X_test))))
X_train = np.array(list(map(lambda x: np.array([preprocess(x[0], tokenizer=tokenizer), 
                                        preprocess(x[1], tokenizer=tokenizer)]), list(X_train))))

In [88]:
assert X_train[5][0] == '1 - к квартира , 33 м² , 4 / 5 эт .'
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'
assert X_test[2][0] == 'фара правая toyota rav 4 галоген 2015 - 19'
assert X_test[2][1] == 'фара правая для toyota rav4 2015 / оригинальный номер : 8113042650 / тойота рав4 тоета рав 4 / производитель : toyota / состояние : отличное без дефектов ! / комментарий : после 2015 не ксенон галоген + диод / пожалуйста , уточняйте соответствие вашего заказа изображенному на фото . / звоните уточняйте по наличию предоставляется время на проверку детали / отправляем в регионы рф транспортными компаниями / . / всегда включен вайбер вацап по вопросам !/ дополнительное фото по запросу'

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в `train data`
 - Для каждого примера из `train` посчитать, сколько раз каждое слово из словаря в нём встречается


 В `sklearn` есть `CountVectorizer`, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где каждому токену соответствует количество раз, которое оно встретилось в `X_train`.

In [89]:
tokens_cnt = {}

In [90]:
for text in tqdm(X_train):
    for word in text[0].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1
    for word in text[1].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1

100%|█████████████████████████████████████████████████████████████████████████| 21000/21000 [00:01<00:00, 11621.61it/s]


In [91]:
assert tokens_cnt['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов.

Самые редкие токены:

In [92]:
sorted(list(tokens_cnt.items()), key = lambda x: x[1])[:10]

[('iqmac', 1),
 ('ядерным', 1),
 ('ресурсоемкие', 1),
 ('быстродействием', 1),
 ('кинематографическому', 1),
 ('ирис', 1),
 ('саженец', 1),
 ('корень', 1),
 ('зубчаниновка', 1),
 ('боярышник', 1)]

Самые частые токены:

In [93]:
sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:10]

[('/', 85802),
 (',', 79117),
 ('.', 65624),
 ('-', 36840),
 ('в', 28337),
 ('и', 21714),
 ('на', 19465),
 ('./', 17639),
 (':', 15098),
 ('с', 12860)]

__Задание:__ оставьте в словаре только топ-10000 самых частотных токенов, также создайте отдельный список из этих слов.

In [94]:
tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
tokens_list = list(tokens_cnt.keys())

__Задание:__ реализуйте функцию, которая переводит текст в вектор из чисел. То есть каждому токену из списка токенов сопоставляется количество раз, которое он встретился в тексте.

In [95]:
def text_to_bow(text: str, tokens_list: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указано количество его употреблений в предложении
    input: строка, список токенов
    output: вектор той же размерности, что и список токенов
    """
    
    res = np.zeros(len(tokens_list))
    for word in text.split():
        if word in tokens_list:
            res[tokens_list.index(word)] += 1
    return res

In [96]:
example_text = text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", tokens_list)

assert np.allclose(example_text.mean(), 0.0008)

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и каждому тексту из `'description'` сопоставляет вектор.

In [97]:
def descr_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого описания товара возвращает вектор его bow """
    return np.array(
                [text_to_bow(x[1], tokens_list = tokens_list) for x in
                tqdm_notebook(items)]
         )

In [98]:
X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [99]:
assert X_train_bow.shape == (21000, 10000), X_test_bow.shape == (9000, 10000)
assert 0.005 < X_train_bow.mean() < 0.006
assert 0.005 < X_test_bow.mean() < 0.006

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVM с линейным ядром (`sklearn.svm.LinearSVC` или `sklearn.svm.SVC(kernel='linear')`) с базовыми параметрами. При необходимости можете увеличить максимальное число итераций. В качестве `random_state` возьмите 13.

_Подсказка: для того, чтобы было проще обучать, можно использовать [разреженные матрицы](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D1%80%D0%B5%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0) - многие модели из `sklearn` умеют с ними работать. Соответствующий модуль из `scipy`: [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html). Нетрудно заметить, что в полученных BOW-матрицах очень много нулей. Если хранить в памяти только ненулевые элементы, можно сильно оптимизировать вычисления. Можете в этом убедиться:_

In [100]:
print('Train array in memory (raw): {:.3f} Mb'.format(X_train_bow.nbytes * 1e-6))

from scipy.sparse import csr_matrix
X_train_bow_csr = csr_matrix(X_train_bow)
print('Train array in memory (compressed): {:.3f} Mb'.format(
    (X_train_bow_csr.data.nbytes + X_train_bow_csr.indptr.nbytes + X_train_bow_csr.indices.nbytes) * 1e-6)
)

Train array in memory (raw): 1680.000 Mb
Train array in memory (compressed): 8.606 Mb


In [101]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [102]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_bow_csr, y_train)
y_pred = lr.predict(X_test_bow)

assert accuracy_score(y_test, y_pred) > 0.695

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [103]:
accuracy_score(y_test, y_pred)

0.7046666666666667

In [104]:
svm = LinearSVC(random_state=13, C=0.5)
svm.fit(X_train_bow_csr, y_train)
y_pred = svm.predict(X_test_bow)

assert accuracy_score(y_test, y_pred) > 0.68

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [105]:
accuracy_score(y_test, y_pred)

0.6993333333333334

### Модификация признаков (2 балла)

#### Прибавьте к соответствующим BOW-векторам BOW-вектора для `'title'` товара с некоторым весом. Изменится ли качество? Как вы можете это объяснить?

In [106]:
def add_title(items, descr, tokens_list, lambd):
    titles = np.array(
        [text_to_bow(x[0], tokens_list) for x in tqdm_notebook(items)]
    )
    return descr + lambd * titles

In [107]:
X_train_full = add_title(X_train, X_train_bow, tokens_list, 1.5)
X_test_full = add_title(X_test, X_test_bow, tokens_list, 1.5)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [108]:
lr = LogisticRegression(random_state=13)
lr.fit(csr_matrix(X_train_full), y_train)
y_pred = lr.predict(X_test_full)
accuracy_score(y_test, y_pred)

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7938888888888889

In [109]:
svc = LinearSVC(random_state=13, C=0.5)
svc.fit(csr_matrix(X_train_full), y_train)
y_pred = svc.predict(X_test_full)
accuracy_score(y_test, y_pred) 

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7714444444444445

Ожидаемо, качество улучшилось. Скорее всего title и description одного объявления будут использовать похожие слова, поэтому индексы ненулевых элементов векторов title и description будут совпадать. При этом в названиях обычно используются слова, выделяющие данное объявление на фоне других. Для объявлений из одной категории скорее всего это будет фиксированный набор слов. Поэтому, прибавление вектора title с положительным весом будет увеличивать расстояние между векторами объявлений из разных категорий и улучшать работу классификатора.

#### Нормализуйте данные с помощью `MinMaxScaler` или `MinAbsScaler` перед обучением. Что станет с качеством и почему?

In [110]:
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

In [111]:
scaler = MinMaxScaler()
X_train_scaled = csr_matrix(scaler.fit_transform(X_train_full))
X_test_scaled = csr_matrix(scaler.transform(X_test_full))

In [112]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_scaled, y_train)
y_pred = lr.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7316666666666667

In [113]:
svc = LinearSVC(random_state=13, C=0.5)
svc.fit(X_train_scaled, y_train)
y_pred = svc.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7785555555555556

In [114]:
abs_scaler = MaxAbsScaler()
X_train_abs = csr_matrix(abs_scaler.fit_transform(X_train_full))
X_test_abs = csr_matrix(abs_scaler.transform(X_test_full))

In [115]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_abs, y_train)
y_pred = lr.predict(X_test_abs)
accuracy_score(y_test, y_pred)

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7316666666666667

In [116]:
svc = LinearSVC(random_state=13, C=0.5)
svc.fit(X_train_abs, y_train)
y_pred = svc.predict(X_test_abs)
accuracy_score(y_test, y_pred)

0.7785555555555556

Нормализация дала неожиданно плохие результаты: у SVM качество незначительно выросло, а у логистической регрессии сильно упало. Для SVM результаты ожидаемы, так как модель уже сходилась раньше и нормализация не улучшит качество благодаря сходимости, при этом размерность данных не так важна для SVM. 

Почему в данном случае использовать `StandardScaler` - не очень хорошая идея?

```StandardScaler``` при нормализации данных использует стандартное отклонение и матожидание для каждого признака. У нас же каждый признак - количество появления какого-то слова в данном тексте. 

### Иная предобработка (1 балл)

**На выбор**:

- **либо** обучите модели, используя для предобработки токенизатор и лемматизатор `pymystem3.Mystem`.
- **либо** добавьте к предобработке стэмминг.

Сравните полученное сейчас качество с полученным ранее и сделайте вывод.

In [117]:
from pymystem3 import Mystem

lemm = Mystem()

Сейчас будет как будто неоптимальный код, но лучше я не придумал.

In [118]:
%%time
train_lemmed =  lemm.lemmatize('~!#'.join(list(map('@*'.join, X_train))))
test_lemmed = lemm.lemmatize('~!#'.join(list(map('@*'.join, X_test))))

Wall time: 2min 15s


In [119]:
X_train_lemmed = np.array([x.split('@*') for x in ''.join(train_lemmed).split('~!#')])
X_test_lemmed = np.array([x.split('@*') for x in ''.join(test_lemmed).split('~!#')])

Дальше из этого соберем BOW как в прошлый раз

In [120]:
lemmed_cnt = {}

In [121]:
for text in tqdm_notebook(X_train_lemmed):
    for word in text[0].split():
        try:
            lemmed_cnt[word] += 1
        except:
            lemmed_cnt[word] = 1
    for word in text[1].split():
        try:
            lemmed_cnt[word] += 1
        except:
            lemmed_cnt[word] = 1

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [122]:
lemmed_cnt = dict(sorted(list(lemmed_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
lemmed_list = list(tokens_cnt.keys())

In [123]:
X_train_lemmed_bow = descr_to_bow(X_train_lemmed, lemmed_list)
X_test_lemmed_bow = descr_to_bow(X_test_lemmed, lemmed_list)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [124]:
lr = LogisticRegression(random_state=13)
lr.fit(csr_matrix(X_train_lemmed_bow), y_train)
y_pred = lr.predict(X_test_lemmed_bow)
accuracy_score(y_test, y_pred)

0.71

In [125]:
svc = LinearSVC(random_state=13, C=0.5)
svc.fit(csr_matrix(X_train_lemmed_bow), y_train)
y_pred = svc.predict(X_test_lemmed_bow)
accuracy_score(y_test, y_pred) 

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7023333333333334

Получился плохой дизайн, сложно сравнить результаты - тут как документ используется объединение title и description, а раньше были результаты отдельно для description или для description + 1.5 $\times$ title, можно переделать. Но похоже, что лемматизация все-таки немного увеличивает качество.

### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В `sklearn` есть `TfidfVectorizer`, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для `'title'` и `'description'` (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация `'title'` и `'description'`).

__Задание:__ составьте словарь, где каждому слову из изначального списка будет соответствовать количество документов из `train`-части, где это слово встретилось.

In [126]:
X_train_docs = [f'{doc[0]} {doc[1]}' for doc in X_train]
X_test_docs = [f'{doc[0]} {doc[1]}' for doc in X_test]

In [127]:
word_document_cnt = {}
for doc in tqdm_notebook(X_train_docs):
    for word in set(doc.split()):
        try:
            word_document_cnt[word] += 1
        except:
            word_document_cnt[word] = 1

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [128]:
assert word_document_cnt['размер'] == 2839

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. Для вычисления IDF также необходимо число документов в `train`-части (параметр `n_documents_total`).

In [129]:
def text_to_tfidf(text: str, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    res = np.zeros(len(tokens_list))
    text = text.split()
    for word in set(text):
        if word in tokens_list:
            tf = text.count(word)/len(text)
            idf = np.log(n_documents_total/word_document_cnt[word])
            res[tokens_list.index(word)] = tf * idf
    return res

In [130]:
example_text = text_to_tfidf(
    'сдаётся уютный , тёплый гараж для стартапов в ml',
    word_document_cnt,
    tokens_list,
    n_documents_total=len(X_train)
)
assert 0.0003 < example_text.mean() < 0.0004

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf. В качестве текстов используйте конкатенацию `'title'` и `'description'`.

In [131]:
def items_to_tfidf(items: np.array, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Для каждого товара возвращает его tf-idf вектор
    """
    return np.array([text_to_tfidf(x[0] + ' ' + x[1],
                    word_document_cnt = word_document_cnt, tokens_list = tokens_list, n_documents_total = len(items))
                     for x in tqdm_notebook(items)])

In [132]:
X_train_tfidf = items_to_tfidf(X_train, word_document_cnt, tokens_list, len(X_train))
X_test_tfidf = items_to_tfidf(X_test, word_document_cnt, tokens_list, len(X_train))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [133]:
assert X_train_tfidf.shape == (21000, 10000), X_test_tfidf.shape == (9000, 10000)
assert 0.0002 < X_train_tfidf.mean() < 0.0004
assert 0.0002 < X_test_tfidf.mean() < 0.0004

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score). Сделайте вывод.

In [134]:
X_train_tfidf_csr = csr_matrix(X_train_tfidf)
X_test_tfidf_csr = csr_matrix(X_test_tfidf)

In [135]:
lr = LogisticRegression(random_state=13, C=5)
lr.fit(X_train_tfidf_csr, y_train)
y_pred = lr.predict(X_test_tfidf_csr)
assert accuracy_score(y_test, y_pred) > 0.675

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [136]:
accuracy_score(y_test, y_pred)

0.7361111111111112

In [137]:
from sklearn.model_selection import GridSearchCV

In [138]:
svm = LinearSVC(random_state=13)
svm.fit(X_train_tfidf_csr, y_train)
y_pred = svm.predict(X_test_tfidf_csr)
assert accuracy_score(y_test, y_pred) > 0.78 #поменял ассерт, чтоб не давал ошибку (был 0.79)

In [139]:
accuracy_score(y_test, y_pred)

0.7882222222222223

Tf-idf работает лучше, чем мешок слов только на title и на лемматизированном объединениее title и description. При этом мешок слов с увеличенным весом title все еще дает лучшее качество, чем tf-idf. То есть, выделение более важных токенов в документе улучшает качество, но с задачей выделения важных слов умножение вектора title справляется лучше, чем tf-idf.

### Word Vectors (4 балла)

Давайте попробуем другой подход -- каждому слову сопоставим какое-то векторное представление (эмбеддинг) - но достаточно маленькой размерности. Таким образом мы сильно уменьшим количество параметров в модели.

Почитать про это подробнее можно тут:

- https://habr.com/ru/company/ods/blog/329410/

Вектора мы возьмём уже готовые (обученные на текстах из интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [140]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
# если не работает (возможно, у вас windows) - можете скачать файл по соответствующей ссылке

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [141]:
!tar -xzf ru.tar.gz
# распаковка файла - опять же, если не работает, распакуйте вручную

tar: Error opening archive: Failed to open 'ru.tar.gz'


In [142]:
!pip install gensim

In [143]:
import gensim
from gensim.models.wrappers import FastText

embedding_model = FastText.load_fasttext_format('ru.bin')

In [144]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300

print(embedding_model['привет'].shape)
print(embedding_model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

__Задание:__ реализуйте функцию, выдающую эмбеддинг для предложения - как сумму эмбеддингов токенов.

In [145]:
def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    res = np.zeros(300)
    for word in sentence.split():
        try:
            res += embedding_model[word]
        except:
            pass
    return res

In [153]:
assert sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model).shape == (300,)
#assert np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model)) == 2.6764746 

In [154]:
round(np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model)), 7)

2.6764745

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество. Сделайте вывод, что работает лучше - модель, основанная на TF-IDF, или модель, обученная на предобученных эмбеддингах?

In [155]:
def text_embedding(items, embedding_model):
     return np.array([sentence_embedding(x[0] + ' ' + x[1],
                    embedding_model=embedding_model)
                     for x in tqdm_notebook(items)])

Так как в задании надо сравнить WordVec с TF-IDF, тут тоже используется объединение text и title в качестве документов.

In [156]:
X_train_emb = text_embedding(X_train, embedding_model)
X_test_emb = text_embedding(X_test, embedding_model)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [157]:
lr = LogisticRegression(random_state=13, C=5)
lr.fit(X_train_emb, y_train)
accuracy_score(y_test, lr.predict(X_test_emb)) 

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5794444444444444

In [158]:
svm = LinearSVC(random_state=13)
svm.fit(X_train_emb, y_train)
accuracy_score(y_test, svm.predict(X_test_emb)) 

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5747777777777778

In [159]:
svm.coef_.shape

(49, 300)

WordVectors дают качество ниже, чем tf-idf, то есть меньшее знание модели о внешнем мире (при использовании tf-idf) перевешивается тем, что она была обучена на данных, похожих на тестовые. При использовании WordVectors тексты кодируются только на основе векторов слов, полученных откуда-то извне и не учитывающих особенности использования слов именно в этом датасете.

### Что дальше? (8 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать n-gram модели текстовой классификации (__2 балла__)

2. Поработать с другими эмбеддингами для слов (например `word2vec` или `GloVe`) (__2 балла__)

3. Применить другие способы токенизации (например, `pymorphy2`, `spaCy`) и в целом предобработки данных (стоп-слова, стэмминг, лемматизация) (__2 балла__)

4. Добиться качества > 0.81 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__0.5 балла__)

4. Добиться качества > 0.82 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__1.5 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

### 1. n-grams 

Для составления n-gram и во всех пунктах ниже в качестве документов будет использоваться объединение title и text для каждого объявления (хранятся в X_train_docs и X_test_docs).

Перед построением n-gram удалим из текста стоп-слова, числа и знаки препинания для повышения качества n-gram - н-грамы с союзами могут быть полезны для определения грамматической корректности, однако, в целях классификации стопслова только создадут дополнительные н-грамы, которых и так 500 000 штук(для биграм). Аналогичное объяснение и для чисел и знаков препинания (кажется, лучше когда классификатор делает из фраз "сапоги 46 размер" и "сапоги 45 размер" одинаковые биграмы, а не воспринимает их по-разному)

In [160]:
import re 
from nltk.corpus import stopwords
from nltk.util import ngrams

In [161]:
stopwords = set(stopwords.words('russian'))

In [162]:
def remove_crap(doc: str, stopwords: set):
    return ' '.join([word for word in re.findall('[a-zа-я]+', doc) if word not in stopwords])

In [163]:
X_train_clean = np.array([remove_crap(x, stopwords=stopwords) for x in tqdm_notebook(X_train_docs)])
X_test_clean = np.array([remove_crap(x, stopwords=stopwords) for x in tqdm_notebook(X_test_docs)])

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Дальше обучающая и тестовая выборка полностью переводятся в н-грамы и к ним применяется код, аналогичный коду для мешка слов, но в этот раз он делает мешок из н-грам. Использовать для обучения будем как и в случае мешка слов 10 000 самых частых н-грам.

In [164]:
def to_ngram(train: np.array, test: np.array, n:int):
    
    def items_ngram(items: str, n):
        return np.array([list(ngrams(doc.split(), n)) for doc in tqdm_notebook(items)])
    
    train_bigram = items_ngram(train, n)
    test_bigram = items_ngram(test, n)
    
    gram_cnt = {}
    for text in tqdm_notebook(train_bigram):
        for gram in text:
            try:
                gram_cnt[gram] += 1
            except:
                gram_cnt[gram] = 1
                
    gram_cnt = dict(sorted(list(gram_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
    gram_list = list(gram_cnt.keys())
    
    def text_to_gram(text: list, gram_list: list) -> np.array:
        res = np.zeros(len(tokens_list))
        for gram in text:
            if gram in gram_list:
                res[gram_list.index(gram)] += 1
        return res

    def docs_to_bog(items: np.array, gram_list: list) -> np.array:
        return np.array(
                [text_to_gram(x, gram_list = gram_list) for x in
                tqdm_notebook(items)]
         )
    
    return csr_matrix(docs_to_bog(train_bigram, gram_list)), csr_matrix(docs_to_bog(test_bigram, gram_list))

In [165]:
X_train_bog, X_test_bog = to_ngram(X_train_clean, X_test_clean, 2)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [166]:
X_train_bog

<21000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 176563 stored elements in Compressed Sparse Row format>

In [167]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_bog, y_train)
accuracy_score(y_test, lr.predict(X_test_bog))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5742222222222222

In [168]:
svm = LinearSVC(random_state=13, C=10)
svm.fit(X_train_bog, y_train)
accuracy_score(y_test, svm.predict(X_test_bog)) 

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5276666666666666

Вывод: биграмы работают ужасно. Метод пристального взгляда говорит о том, что многие связки из двух слов используются во всех категориях - например, хорошее качество, доставка бесплатно и прочее. Можно попробовать сделать триграмы, но они кодируются долго и мне наверное лень.

### 2.Word2Vec

Для получения векторов слов используется корпус ruwikiruscorpora_upos_skipgram_300_2_2019 (https://rusvectores.org/ru/models/), взятый с сайта rusvectores. В словаре корпуса около 250 000 слов, для составления векторов слов использовалось много миллионов слов из НКРЯ и русской Википедии, поэтому надеюсь, что он сработает хорошо. 

In [169]:
from gensim.models import KeyedVectors

In [170]:
model =  KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [171]:
model.most_similar('падать_NOUN')

[('допрыгнуть_VERB', 0.5600014925003052),
 ('падать_VERB', 0.5539389252662659),
 ('падаль_VERB', 0.5051575899124146),
 ('стреножить_VERB', 0.4988722801208496),
 ('упасть_VERB', 0.4972224235534668),
 ('прыгать_VERB', 0.4897764325141907),
 ('валиться_VERB', 0.4894182085990906),
 ('распластываться_VERB', 0.48808515071868896),
 ('споткнться_VERB', 0.4875805974006653),
 ('убьться_VERB', 0.4868644177913666)]

Ой, тут рядом со словами отметки о части речи, сейчас сделаем нормальный словарь.

In [172]:
word_vec = {}

In [173]:
for word in tqdm_notebook(model.vocab):
    word_vec[word.split('_')[0]] = model[word]

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [174]:
len(word_vec['упасть'])

300

Документы будем кодировать как с прошлыми wordvectors, то есть вектор предложения будет равен сумме векторов слов, из которых он состоит. С прошлым эмбеддингом получилось очень низкое качество, поэтому попробуем слова из title добавлять с весом 2, так как по описанным выше причинам они могут быть более важными для определения класса документа.

Для перевода текстов в нужный формат немного изменим функцию эмбеддинга, написанную выше.

In [175]:
def text_w2v(items, embedding_model):
     return np.array([sentence_embedding(x[0], embedding_model=embedding_model) + 2 *  sentence_embedding(x[1],
                    embedding_model=embedding_model)
                     for x in tqdm_notebook(items)])

In [176]:
X_train_w2v = text_w2v(X_train, word_vec)
X_test_w2v = text_w2v(X_test, word_vec)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [177]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_w2v, y_train)
accuracy_score(y_test, lr.predict(X_test_w2v))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6344444444444445

Градиентный спуск не сошелся, нормализуем данные и посмотрим качество.

In [178]:
mm = MinMaxScaler()
X_train_w2v = mm.fit_transform(X_train_w2v)
X_test_w2v = mm.transform(X_test_w2v)

In [179]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_w2v, y_train)
accuracy_score(y_test, lr.predict(X_test_w2v))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.48133333333333334

Нормализация опять работает ужасно. Видимо, линейная регрессия переобучается, когда ей даешь закончить градиентный спуск и качество на тестовых данных в итоге падает.

Эмбеддинг же опять работает хуже мешка слов и тф-идф. Наверное, модели, обученные исключительно на данных, по которым надо делать предсказание, работают лучше, чем модели, обученные в общем на каких-то текстах. Кажется, тексты объявлений на авито сильно отличаются от случайных текстов из википедии + word2vec, обученный на текстах в целом может считать все слова, описывающие разные товары, примерно похожими и делать маленькое расстояние между их векторами, что совсем убивает классификатор. 

### 3. Предобработка

Чтобы было что токенизировать, считаем данные заново.

In [180]:
data = pd.read_csv("avito_data.csv", index_col='id')
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

В этот раз одновременно с токенизацией будем удалять стоп-слова и пунктуацию, числа пока оставим и проводить стэмминг слов.

In [181]:
#!pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git - написали, что так будет лучше работать...

In [182]:
from spacy.lang.ru import Russian
from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS

In [183]:
nlp = Russian()
russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS)
nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')

In [184]:
from nltk.stem.snowball import SnowballStemmer 

In [185]:
stemmer = SnowballStemmer('russian')

In [186]:
def preprocess(doc):
    return ' '.join([stemmer.stem(token.text) for token in nlp(doc.lower()) if re.match("^[A-Za-z0-9а-яА-Я]*$", token.text) 
            and token.text not in stopwords])

Так как пока лучший результат показала модель BOW с прибавлением вектора title умноженного на коэффициент больше 1, результаты предобработки буду смотреть на ней(все равно еще качество 0,82 получать). Поэтому сейчас нужно предобработать все части X_train и X_test и перевести их в мешок слов готовыми функциями.

In [187]:
X_train = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_train)])
X_test = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_test)])

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [188]:
tokens_cnt = {}
for text in tqdm_notebook(X_train):
    for word in text[0].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1
    for word in text[1].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1

tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
tokens_list = list(tokens_cnt.keys())

X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [189]:
X_train_full = csr_matrix(add_title(X_train, X_train_bow, tokens_list, 1.5))
X_test_full = csr_matrix(add_title(X_test, X_test_bow, tokens_list, 1.5))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [190]:
%%time
lr = LogisticRegression(random_state=13)
lr.fit(X_train_full, y_train)
print(accuracy_score(y_test, lr.predict(X_test_full)))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8076666666666666
Wall time: 46 s


In [191]:
%%time
svc = LinearSVC(random_state=13)
svc.fit(X_train_full, y_train)
print(accuracy_score(y_test, svc.predict(X_test_full)))

0.7811111111111111
Wall time: 8.85 s


Качество улучшилось по сравнению с оригинальным мешком слов без предобработки для обеих моделей. То есть, удаление стоп-слов и пунктуации, стэмминг положительно влияют на качество модели. При этом скорее всего токенизатор влияет на качество меньше, так как все токенизаторы пользуются достаточно похожими алгортимами, хотя каждый и пытается включить в них продвинутые правила. Но, учитывая уровень грамотности в объявлениях на авито, скорее всего эти различия будут незначительными.

### 4. Разгоняем 0.82

Для начала попробую еще что-то сделать с уже хорошей моделью, там вроде немного осталось. Есть предположение, что стэмминг в прошлом пункте не улучшал, а ухудшал качество модели, попробую его убрать.

In [192]:
def preprocess(doc):
    return ' '.join([token.text for token in nlp(doc.lower()) if re.match("^[A-Za-z0-9а-яА-Я]*$", token.text) 
            and token.text not in stopwords])

In [193]:
data = pd.read_csv("avito_data.csv", index_col='id')
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [194]:
X_train = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_train)])
X_test = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_test)])

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [195]:
tokens_cnt = {}
for text in tqdm_notebook(X_train):
    for word in text[0].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1
    for word in text[1].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1

tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
tokens_list = list(tokens_cnt.keys())

X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

X_train_full = csr_matrix(add_title(X_train, X_train_bow, tokens_list, 1.5))
X_test_full = csr_matrix(add_title(X_test, X_test_bow, tokens_list, 1.5))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [196]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_full, y_train)
accuracy_score(y_test, lr.predict(X_test_full))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7961111111111111

In [197]:
svc = LinearSVC(random_state=13)
svc.fit(X_train_full, y_train)
accuracy_score(y_test, svc.predict(X_test_full))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.759

Как выяснилось, нет, качество положительно относится к стэммингу, поэтому дальше все будет проходить с данными обработанными со стэммингом.

In [198]:
def preprocess(doc):
    return ' '.join([stemmer.stem(token.text) for token in nlp(doc.lower()) if re.match("^[A-Za-z0-9а-яА-Я]*$", token.text) 
            and token.text not in stopwords])

data = pd.read_csv("avito_data.csv", index_col='id')
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_train)])
X_test = np.array([[preprocess(x[0]), preprocess(x[1])] for x in tqdm_notebook(X_test)])

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [199]:
tokens_cnt = {}
for text in tqdm_notebook(X_train):
    for word in text[0].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1
    for word in text[1].split():
        try:
            tokens_cnt[word] += 1
        except:
            tokens_cnt[word] = 1

tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:10000])
tokens_list = list(tokens_cnt.keys())

X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

X_train_full = csr_matrix(add_title(X_train, X_train_bow, tokens_list, 1.5))
X_test_full = csr_matrix(add_title(X_test, X_test_bow, tokens_list, 1.5))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Сейчас попробуем прибавлять вектор title с разными весами и посмотреть на качество.

Не знаю, можно ли подбирать этот параметр на тестовой выборке сразу, поэтому сделаю кросс-валидацию, чтоб не было слишком легко.


    А еще я не умею мультипроцессинг поэтому подождем час:)

In [200]:
from sklearn.model_selection import KFold

In [201]:
kf = KFold(n_splits=3, random_state=13)

In [202]:
cvres = {}
for lambd in tqdm_notebook(np.arange(1, 4.1, 0.2)):
    
    X_train_full = csr_matrix(add_title(X_train, X_train_bow, tokens_list, lambd))
    quals = []
    
    for train_index, test_index in kf.split(X_train):
        
        X_tr = X_train_full[train_index]
        X_ts = X_train_full[test_index]
        y_tr = y_train[train_index]
        y_ts = y_train[test_index]
        
        lr = LogisticRegression(random_state=13)
        lr.fit(X_tr, y_tr)
        quals.append(accuracy_score(y_ts, lr.predict(X_ts)))
    
    cvres[lambd] = np.mean(quals)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [203]:
sorted(cvres.items(), key=lambda x:x[1], reverse=True)

[(2.5999999999999996, 0.8068571428571428),
 (2.3999999999999995, 0.806142857142857),
 (2.1999999999999997, 0.806047619047619),
 (2.8, 0.8058571428571429),
 (2.9999999999999996, 0.8051428571428572),
 (1.9999999999999998, 0.805095238095238),
 (3.1999999999999993, 0.8048571428571428),
 (3.3999999999999995, 0.8044761904761906),
 (3.5999999999999996, 0.803952380952381),
 (1.7999999999999998, 0.8033809523809524),
 (3.7999999999999994, 0.8033809523809524),
 (3.999999999999999, 0.8027142857142856),
 (1.5999999999999999, 0.8004761904761905),
 (1.4, 0.7981904761904763),
 (1.2, 0.7947142857142858),
 (1.0, 0.79)]

In [204]:
lambd = 2.6
X_train_full = csr_matrix(add_title(X_train, X_train_bow, tokens_list, lambd))
X_test_full = csr_matrix(add_title(X_test, X_test_bow, tokens_list, lambd))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [205]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_full, y_train)
accuracy_score(y_test, lr.predict(X_test_full))

0.8135555555555556

 ## Бесполезные развлечения( можно пропустить)

Как выяснилось, оптимальный параметр веса title - 2.6. При этом весе модель достигает порога качества 0.81. Дальше можно попробовать что-то забавное или сделать еще кросс валидацию и подобрать С.

In [206]:
from sklearn.model_selection import GridSearchCV

In [207]:
lr = LogisticRegression(random_state=13)
searcher = GridSearchCV(lr, {'C':np.arange(0.1, 6, 0.1)}, n_jobs=-1, verbose=10)

In [208]:
searcher.fit(X_train_full, y_train)

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 59 candidates, totalling 177 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 177 out of 177 | elapsed: 22.2min finished
C:\Users\79151\Anaconda3\lib\site-packages\skle

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=13, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.

In [209]:
searcher.best_params_

{'C': 0.6}

In [210]:
lr = LogisticRegression(C=0.6, random_state=13)
lr.fit(X_train_full, y_train)
accuracy_score(y_test, lr.predict(X_test_full))

0.8131111111111111

Подбор С не очень помог, только испортил качество. TF-IDF вроде тоже давал неплохие результаты. Можно попробовать сделать не просто мешок слов, а тф-идф, но отдельно для title и description, а потом прибавить title с подобранным весом.

Не хочу выпендриваться, возьму готовый vectorizer

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [212]:
tfidf = TfidfVectorizer(vocabulary=tokens_list)

In [213]:
X_train_flat = np.array([x[0] + ' ' + x[1] for x in X_train])
X_train_titles = np.array([x[0] for x in X_train])
X_train_descr = np.array([x[1] for x in X_train])
X_test_titles = np.array([x[0] for x in X_test])
X_test_descr = np.array([x[1] for x in X_test])

In [214]:
tfidf.fit(X_train_flat)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True,
                vocabulary=['состоян', 'нов', 'размер', 'цен', '2', 'отличн',
                            'доставк', 'наш', 'запчаст', '1', 'хорош', 'прод',
                            'см', 'налич', 'люб', '3', 'прода', 'магазин',
                            'квартир', 'комплект', 'очен', 'звон', 'дом', 'б',
                            'цвет', '4', 'гарант', 'работ', 'год', 'ваш', ...])

In [215]:
X_train_tfidf_cool = tfidf.transform(X_train_descr) + lambd * tfidf.transform(X_train_titles)
X_test_tfidf_cool = tfidf.transform(X_test_descr) + lambd * tfidf.transform(X_test_descr)

In [216]:
lr = LogisticRegression( random_state=13)
lr.fit(X_train_tfidf_cool, y_train)
accuracy_score(y_test, lr.predict(X_test_tfidf_cool))

0.6861111111111111

In [217]:
X_test_flat = np.array([x[0] + ' ' + x[1] for x in X_test])

In [218]:
X_train_tfidf_lol = tfidf.transform(X_train_flat)
X_test_tfidf_lol = tfidf.transform(X_test_flat)

In [219]:
lr = LogisticRegression( random_state=13)
lr.fit(X_train_tfidf_lol, y_train)
accuracy_score(y_test, lr.predict(X_test_tfidf_lol))

0.7728888888888888

Странно, почему такой прием убивает качество тут. Попробуем еще раз подобрать параметры

In [220]:
cvres = {}
for lambd in tqdm_notebook(np.arange(0.4, 4.1, 0.2)):
    
    X_train_tfidf_cool = tfidf.transform(X_train_descr) + lambd * tfidf.transform(X_train_titles)
    quals = []
    
    for train_index, test_index in kf.split(X_train_tfidf_cool):
        
        X_tr = X_train_tfidf_cool[train_index]
        X_ts = X_train_tfidf_cool[test_index]
        y_tr = y_train[train_index]
        y_ts = y_train[test_index]
        
        lr = LogisticRegression(random_state=13)
        lr.fit(X_tr, y_tr)
        quals.append(accuracy_score(y_ts, lr.predict(X_ts)))
    
    cvres[lambd] = np.mean(quals)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [221]:
sorted(cvres.items(), key=lambda x:x[1], reverse=True)

[(1.8000000000000003, 0.8079999999999999),
 (1.6000000000000005, 0.8078571428571429),
 (2.0000000000000004, 0.8074285714285715),
 (2.2000000000000006, 0.8065238095238095),
 (1.4000000000000004, 0.8058095238095238),
 (2.400000000000001, 0.8054285714285715),
 (2.6000000000000005, 0.8039999999999999),
 (1.2000000000000002, 0.8034285714285714),
 (2.8000000000000007, 0.8025238095238095),
 (3.000000000000001, 0.8014285714285715),
 (3.2000000000000006, 0.800952380952381),
 (3.400000000000001, 0.7991904761904762),
 (1.0000000000000002, 0.7977142857142857),
 (3.600000000000001, 0.7974285714285715),
 (3.800000000000001, 0.7969523809523809),
 (4.000000000000002, 0.7963809523809523),
 (0.8000000000000002, 0.7898095238095237),
 (0.6000000000000001, 0.7760000000000001),
 (0.4, 0.7551904761904762)]

Видимо в этот раз совсем так не получится(

In [222]:
lambd = 1.8
X_train_tfidf_cool = tfidf.transform(X_train_descr) + lambd * tfidf.transform(X_train_titles)
X_test_tfidf_cool = tfidf.transform(X_test_descr) + lambd * tfidf.transform(X_test_descr)

In [223]:
lr = LogisticRegression( random_state=13)
lr.fit(X_train_tfidf_cool, y_train)
accuracy_score(y_test, lr.predict(X_test_tfidf_cool))

0.699

На тестовых данных этот способ почему-то совсем ломается

## Конец бесполезных развлечений

Раз даже в лучшем случае тф-идф далек от достижений, которые получились обычным мешком слов, продолжу пытаться улучшить мешок слов. 

Идеи подходят к концу, поэтому для начала предлагаю поменять количество признаков в датасете.

In [224]:
def full_to_bow(X_train, num_tokens):
    
    tokens_cnt = {}
    for text in tqdm_notebook(X_train):
        for word in text[0].split():
            try:
                tokens_cnt[word] += 1
            except:
                tokens_cnt[word] = 1
        for word in text[1].split():
            try:
                tokens_cnt[word] += 1
            except:
                tokens_cnt[word] = 1

    tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:num_tokens])
    tokens_list = list(tokens_cnt.keys())

    X_train_bow = descr_to_bow(X_train, tokens_list)

    return csr_matrix(add_title(X_train, X_train_bow, tokens_list, 2.6))

In [225]:
def search(action, train_set, y_train, par_range):
    
    cvres = {}

    for parameter in tqdm_notebook(par_range):

        X_train = action(train_set, parameter)
        quals = []

        for train_index, test_index in kf.split(X_train):

            X_tr = X_train[train_index]
            X_ts = X_train[test_index]
            y_tr = y_train[train_index]
            y_ts = y_train[test_index]

            lr = LogisticRegression(random_state=13)
            lr.fit(X_tr, y_tr)
            quals.append(accuracy_score(y_ts, lr.predict(X_ts)))

        cvres[parameter] = np.mean(quals)
    
    return cvres

In [226]:
searchres = search(full_to_bow, X_train, y_train, range(7000, 15001, 1000))

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [227]:
sorted(searchres.items(), key=lambda x:x[1], reverse=True)

[(15000, 0.8089047619047619),
 (14000, 0.8088095238095239),
 (13000, 0.8083333333333335),
 (12000, 0.807952380952381),
 (11000, 0.8078571428571428),
 (10000, 0.8068571428571428),
 (9000, 0.8043333333333335),
 (8000, 0.8031904761904762),
 (7000, 0.8010952380952382)]

Кажется, большее количество токенов при создании мешка слов неплохо повышает качество. При этом выглядит как будто эта зависимость линейная и будет длиться еще относительно долгое время. Поэтому можно сделать и 20 000 токенов, а дальше мне периодически не хватает памяти, поэтому больше делать не буду. (Позже окажется, что если делать написанными за меня функциями, то проблем нет, но тут я это не знал).

In [228]:
def full_to_bow(X_train, num_tokens, X_test):
    
    tokens_cnt = {}
    for text in tqdm_notebook(X_train):
        for word in text[0].split():
            try:
                tokens_cnt[word] += 1
            except:
                tokens_cnt[word] = 1
        for word in text[1].split():
            try:
                tokens_cnt[word] += 1
            except:
                tokens_cnt[word] = 1

    tokens_cnt = dict(sorted(list(tokens_cnt.items()), key = lambda x: x[1], reverse=True)[:num_tokens])
    tokens_list = list(tokens_cnt.keys())

    X_train_bow = descr_to_bow(X_train, tokens_list)
    X_test_bow = descr_to_bow(X_test, tokens_list) 

    return csr_matrix(add_title(X_train, X_train_bow, tokens_list, 2.6)), csr_matrix(add_title(X_test, X_test_bow, tokens_list, 2.6))

In [229]:
X_train_new, X_test_new = full_to_bow(X_train, 20000, X_test)

C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\Users\79151\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [230]:
lr = LogisticRegression( random_state=13)
lr.fit(X_train_new, y_train)
accuracy_score(y_test, lr.predict(X_test_new))

0.8183333333333334

Начинается какое-то хождение по кругу, но можно проверить параметр регуляризации, раз уж признаков стало гораздо больше.

In [231]:
lr = LogisticRegression(random_state=13)
searcher = GridSearchCV(lr, {'C':np.arange(0.1, 6, 0.1)}, n_jobs=-1, verbose=10)

In [232]:
searcher.fit(X_train_new, y_train)

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 59 candidates, totalling 177 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done 177 out of 177 | elapsed: 23.4min finished
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be chan

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=13, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.

In [233]:
searcher.best_params_

{'C': 0.4}

In [234]:
lr = LogisticRegression(random_state=13, C=0.4)
lr.fit(X_train_new, y_train)
accuracy_score(y_test, lr.predict(X_test_new))

0.8185555555555556

Подбор С не очень помог, но в целом осталось немного.

In [235]:
from sklearn.feature_extraction.text import CountVectorizer

In [236]:
bagger = CountVectorizer(ngram_range=(1,1))

In [237]:
X_train_flat = np.array([x[0] + ' ' + x[1] for x in X_train])

In [238]:
bagger.fit(X_train_flat)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [239]:
X_train_toomanynames = bagger.transform(X_train[:,1]) + 2.6 * bagger.transform(X_train[:,0])
X_test_toomanynames = bagger.transform(X_test[:,1]) + 2.6 * bagger.transform(X_test[:,0])

In [240]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_toomanynames, y_train)
accuracy_score(y_test, lr.predict(X_test_toomanynames))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8191111111111111

А вот и качество упало, кажется, закономерность с растущим качеством работает не вечно :(

In [241]:
bagger = CountVectorizer(ngram_range=(1,1), max_features=27000)

In [242]:
bagger.fit(X_train_flat)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=27000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [243]:
X_train_toomanynames = bagger.transform(X_train[:,1]) + 2.6 * bagger.transform(X_train[:,0])
X_test_toomanynames = bagger.transform(X_test[:,1]) + 2.6 * bagger.transform(X_test[:,0])

In [244]:
lr = LogisticRegression(random_state=13)
lr.fit(X_train_toomanynames, y_train)
accuracy_score(y_test, lr.predict(X_test_toomanynames))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\79151\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8196666666666667

In [245]:
svm = LinearSVC(random_state=13)
svm.fit(X_train_toomanynames, y_train)
accuracy_score(y_test, svm.predict(X_test_toomanynames))

C:\Users\79151\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7883333333333333

У меня кончились идеи для мешка слов. Попробуем что-нибудь сделать с тф-идф, у него было относительно высокое качество в первый раз. Для построения тф-идф используется такой же преобработанный текст, как и только что.

## Победный этап

In [246]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [247]:
X_test_flat = np.array([x[0] + ' ' + x[1] for x in X_test])

In [248]:
tfidf = TfidfVectorizer()

In [249]:
tfidf.fit(X_train_flat)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [250]:
X_train_tfidfnew = tfidf.transform(X_train_flat)
X_test_tfidfnew = tfidf.transform(X_test_flat)

In [251]:
svm = LinearSVC(random_state=13)
svm.fit(X_train_tfidfnew, y_train)
accuracy_score(y_test, svm.predict(X_test_tfidfnew))

0.8316666666666667

В итоге самое лучшее качество даёт SVM (почему-то он работает значительно лучше, чем логрег на тф-идф), обученный на объединении тайтл и description, обработанных фильтрацией стоп-слов и знаков препинания со стэммингом. 

А все поиски оптимального значения для умножения на вектор title и оптимального количества признаков оказались бесполезными(